# farmácias populares

<p align="justify">No [portal brasileiro de dados abertos](http://dados.gov.br), há um conjunto de dados com informações sobre os estabelecimentos que aderiram ao programa [*Aqui tem Farmácias Popular*](http://dados.gov.br/dataset/farmacia_popular_estabelecimento). Usaremos a biblioteca [pandas](https://pandas.pydata.org) do Python para algumas análises descritivas e a biblioteca [folium](https://github.com/python-visualization/folium) para plotar a localização dos estabelecimentos. Todo o documento foi gerado no [Jupyter](http://jupyter.org).</p>

## lendo os dados

In [135]:
import pandas

In [147]:
# lendos os dados da url

dados = pandas.read_csv("http://i3geo.saude.gov.br/i3geo/ogc.php?service=WFS&version=1.0.0&request=GetFeature&typeName=farmacia_popular_estabelecimento&outputFormat=CSV")

In [137]:
import sqlite3

In [138]:
# criando uma base em sqlite e estabelecendo conecção

conn = sqlite3.connect("farmacia_pop.db")

In [139]:
# salvando a tabela

dados.to_sql("estabelecimentos", conn, if_exists = "replace")

## consultas via SQL

### quantitativo por estados do sudeste

Na tabela abaixo, através de uma query, verica-se o número de estabelecimentos credenciados ao programa nos 4 estados da região sudeste.

In [140]:
import tabulate

In [141]:
print(tabulate.tabulate(pandas.read_sql_query("SELECT uf, COUNT(*) AS n, COUNT(uf) * 100.0 /(SELECT COUNT(*) FROM estabelecimentos WHERE uf IN ('SP', 'RJ', 'MG', 'ES')) AS '%' FROM estabelecimentos WHERE uf IN ('SP', 'RJ', 'MG', 'ES') GROUP BY uf", conn, index_col = ['uf']), headers = ['uf', 'n', '%'], tablefmt="grid"))

+------+------+----------+
| uf   |    n |        % |
+======+======+==========+
| ES   |  922 |  7.30181 |
+------+------+----------+
| MG   | 5155 | 40.8252  |
+------+------+----------+
| RJ   | 1897 | 15.0234  |
+------+------+----------+
| SP   | 4653 | 36.8496  |
+------+------+----------+


## trabalhando com mapas

Abaixo temos a distribuição espacial das farmácias populares no centro do Rio de Janeiro.

In [142]:
import folium

In [143]:
# filtrando estabelecimentos no centro do Rio de Janeiro

dados_rj = dados[(dados['no_cidade'] == 'RIO DE JANEIRO') & 
                 (dados['no_bairro_farmacia'] == 'CENTRO')]

In [144]:
# mapa

map_osm = folium.Map(location = [-22.9005452, -43.1956282], zoom_start = 14)

In [145]:
# marcando os pontos onde se encontram os estabelecimentos

for _, x in dados_rj.iterrows():
    folium.Marker(location = [x['lat'], x['long']], popup = x['ds_endereco_farmacia']).add_to(map_osm)

In [146]:
map_osm

O número destes estabelecimentos no centro do Rio é de 53.

In [90]:
dados[(dados['no_cidade'] == 'RIO DE JANEIRO')].groupby('no_bairro_farmacia').size().sort_values(ascending=False).head(5)

no_bairro_farmacia
CENTRO             53
CAMPO GRANDE       47
TIJUCA             43
COPACABANA         33
BARRA DA TIJUCA    26
dtype: int64

A JAMYR VASCONCELLOS S.A. é a empresa que possui mais estabelecimentos no centro do Rio.

In [96]:
dados[(dados['no_cidade'] == 'RIO DE JANEIRO') & 
      (dados['no_bairro_farmacia'] == 'CENTRO')].groupby('no_farmacia').size().sort_values(ascending=False).head(5)

no_farmacia
JAMYR VASCONCELLOS S.A.            8
DROGARIAS PACHECO SA               5
SENDAS DISTRIBUIDORA S/A           4
DROGARIA SÃO PAULO S.A.            3
EMPREENDIMENTOS PAGUE MENOS S/A    2
dtype: int64

Considerando todo o país a RAIA DROGASIL S/A é que apresenta mais estabelecimentos vinculados ao programa.

In [97]:
dados.groupby('no_farmacia').size().sort_values(ascending=False).head(5)

no_farmacia
RAIA DROGASIL S/A                       439
COMERCIO DE MEDICAMENTOS BRAIR LTDA     344
EMPREENDIMENTOS PAGUE MENOS S/A         281
CIA LATINO AMERICANA DE MEDICAMENTOS    245
EMPREENDIMENTOS PAGUE MENOS S.A.        235
dtype: int64